In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers
!pip install -q gradio
!pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.3 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 212 kB 30.1 MB/s 
     |████████████████████████████████| 272 kB 38.4 MB/s 
     |████████████████████████████████| 140 kB 61.8 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |███████████████████████████████

In [ ]:
from transformers import pipeline
import gradio as gr
import tensorflow as tf
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

In [ ]:
model_name = "EleutherAI/gpt-neo-1.3B"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

In [ ]:
model = GPTNeoForCausalLM.from_pretrained(model_name, pad_token_id = tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

In [ ]:
free_vram = 0.0
if torch.cuda.is_available():
    from pynvml import *
    nvmlInit()
    h = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(h)
    free_vram = info.free/1048576000
    print("There is a GPU with " + str(free_vram) + "GB of free VRAM")

In [ ]:
if model_name == "EleutherAI/gpt-neo-2.7B" and free_vram>13.5:
    use_cuda = True
    model.to("cuda:0")
elif model_name == "EleutherAI/gpt-neo-1.3B" and free_vram>7.5:
    use_cuda = True
    model.to("cuda:0")
else:
    use_cuda = False

In [ ]:
def generate_text(input):
  input_ids = tokenizer(input, return_tensors="pt").input_ids
  if use_cuda:
    input_ids = input_ids.cuda()
  gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=500)
  output = tokenizer.batch_decode(gen_tokens)[0]
  return '.'.join(output.split('.')[:-1]) + '.'

In [ ]:
output_text = gr.outputs.Textbox()
out = gr.Interface(generate_text, 'textbox', output_text, title='GPT-Neo',
             description = 'Very first project in GPT-Neo').launch(debug=True)

out.launch()